In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from PIL import Image

# Create necessary folders if needed
os.makedirs('content', exist_ok=True)

# Step 1: Load and encode your known face - USING YOUR CORRECT PATH
def load_and_encode_known_face(image_path):
    """Load your known photo and extract face encoding"""
    print(f"Loading known face from: {image_path}")
    
    # Check if file exists
    if not os.path.exists(image_path):
        print(f"❌ Error: File not found at {image_path}")
        print("Please check the file path and try again")
        return None
    
    # Load image
    known_image = face_recognition.load_image_file(image_path)
    
    # Get face encodings
    known_encodings = face_recognition.face_encodings(known_image)
    
    if len(known_encodings) > 0:
        print("✅ Known face encoded successfully!")
        return known_encodings[0]
    else:
        print("❌ No face found in the known image")
        print("Make sure the photo shows a clear face")
        return None

# Load your known face - USING YOUR ACTUAL PATH
known_face_encoding = load_and_encode_known_face('/Users/ithelp/Downloads/Code/opencv/geeksforgeeks/face_det/img/1.jpg')

if known_face_encoding is None:
    print("Please check your image path and make sure the file contains a face")
    exit()

# Step 2: Real-time recognition
def recognize_faces():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    print("Starting face recognition... Press 'q' to quit")
    print("Looking for YOUR face...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Convert BGR to RGB (face_recognition uses RGB)
        rgb_frame = frame[:, :, ::-1]
        
        # Find all faces in current frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Compare with known face
            matches = face_recognition.compare_faces([known_face_encoding], face_encoding)
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)
            
            # Get face coordinates
            top, right, bottom, left = face_location
            
            if matches[0]:
                # It's a match! (You)
                color = (0, 255, 0)  # Green
                label = "YOU! ({:.2f})".format(distance[0])
                status = "✅ MATCH FOUND! It's YOU!"
            else:
               